In [114]:
import numpy as np
from numpy import linalg as LA
from ncon import ncon
from scipy.linalg import null_space as NS

In [200]:
N = 3
mps = np.random.random(2**N) #+ np.random.random(2**N) * 1j
mps_norm = mps / LA.norm(mps)
psi_norm = mps_norm / LA.norm(mps_norm)
psi = psi_norm.reshape(2,2,2)
print(LA.norm(psi))


1.0


In [201]:
B_sites = [0+0j for _ in range(0,N)]

U,S,Vh = LA.svd(psi.reshape(4,2), full_matrices=False)

B_sites[2] = Vh.reshape(2,2)
temp = ((U[:,:2]@np.diag(S[:2])).reshape(2,-1))/LA.norm(S)
U,S,Vh = LA.svd(temp.reshape(2,4), full_matrices=False)
B_sites[1] = Vh.reshape(2,2,2)
temp = ((U[:,:2]@np.diag(S[:2])).reshape(2,-1))/LA.norm(S)
B_sites[0] = temp

CHECK CONDIZIONI RIGHT-CANONICAL FORM


In [202]:
print(np.round(ncon([B_sites[2],B_sites[2].conjugate()],[[-1,1],[-2,1]]))) #A_N@A_N* = I_2
print(np.round(ncon([B_sites[1],B_sites[1].conjugate()],[[-1,1,2],[-2,1,2]]))) #A_n@A_n* = I_2
print(np.round(ncon([B_sites[0],B_sites[0].conjugate()],[[1,2],[1,2]]))) #A_1@A_1 = 1


[[ 1. -0.]
 [-0.  1.]]
[[ 1. -0.]
 [-0.  1.]]
1.0


CALCOLO NULL SPACE DEI VARI TENSORI

In [216]:
null_K = [0+0j for _ in range(0,N)] 
print("Tensor 3 does not need addition because is a 2x2 matrix")
print(B_sites[2])
null_K[2] = NS(B_sites[2])
print(null_K[2])

print("\nTensor 2 needs 2 2x2-vectors from the null space")
print(B_sites[1].reshape(4,2))
null_K[1] = NS(B_sites[1].reshape(2,4))
print("First: ",null_K[1][:,0].reshape(4,1))
print("Second: ",null_K[1][:,1].reshape(4,1))
print("\nTensor 1 needs 3 2x2-vectors from the null space")
print(B_sites[0].reshape(4,1))
null_K[0] = NS(B_sites[0].reshape(1,4))
print("First:",null_K[0][:,0].reshape(4,1))
print("Second: ",null_K[0][:,1].reshape(4,1))
print("Third: ",null_K[0][:,2].reshape(4,1))

Tensor 3 does not need addition because is a 2x2 matrix
[[-0.66840003 -0.74380199]
 [-0.74380199  0.66840003]]
[]

Tensor 2 needs 2 2x2-vectors from the null space
[[ 0.42296613 -0.31447836]
 [ 0.83034424  0.18091838]
 [ 0.49865413 -0.6379433 ]
 [-0.40259031 -0.42695837]]
First:  [[-0.74673129]
 [-0.52854114]
 [ 0.24938929]
 [-0.31755572]]
Second:  [[-0.12180355]
 [-0.46343929]
 [-0.2936911 ]
 [ 0.8271236 ]]

Tensor 1 needs 3 2x2-vectors from the null space
[[-0.68207656]
 [-0.22483073]
 [-0.65533529]
 [ 0.23400506]]
First: [[-0.22483073]
 [ 0.96994854]
 [-0.08759382]
 [ 0.03127772]]
Second:  [[-0.65533529]
 [-0.08759382]
 [ 0.74468205]
 [ 0.09116813]]
Third:  [[0.23400506]
 [0.03127772]
 [0.09116813]
 [0.96744597]]


CREAZIONE DELLE MATRICI G

In [214]:
G = [0+0j for _ in range(N)]
G[0] = np.block([[B_sites[0].reshape(4,1).T], [null_K[0].T]])
G[1] = np.block([[B_sites[1].reshape(4,2).T], [null_K[1].T]])
G[2] = B_sites[2]

for g in G:
    print(ncon([g,g.T],[[-1,1],[-2,1]]))


[[ 1.00000000e+00  7.38262981e-18  7.99352900e-18 -7.92456087e-18]
 [-1.69011903e-17  1.00000000e+00 -3.42315099e-18  1.61156161e-17]
 [-1.21707750e-17 -5.54648553e-17  1.00000000e+00  5.38415293e-17]
 [ 1.71203474e-17  1.26461691e-17  2.60859537e-17  1.00000000e+00]]
[[-0.40554866  0.42444897 -0.07620255 -1.01614691]
 [ 0.33846866  0.30665473 -0.30593395 -0.10120348]
 [-0.29717466 -0.70031153  0.12027694  0.18443902]
 [ 0.2127846  -0.41307731 -0.08125211  1.02430296]]
[[ 1.00000000e+00 -2.17644345e-18]
 [-2.17644345e-18  1.00000000e+00]]


CHECK UNITARIETA'

In [206]:
G_rank4 = [0+0j for _ in range(N)]
G_rank4[0] = G[0].reshape(2,2,2,2)
G_rank4[1] = G[1].reshape(2,2,2,2)
G_rank4[2] = G[2].reshape(2,2) 